<a href="https://colab.research.google.com/github/JhonatanGttg/Algoritmos-e-Complexidade/blob/main/G1_Estra%C3%A7%C3%A3o_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# G1 - Extração de Manchetes

## 1) Instalar dependências

In [1]:
%pip -q install requests beautifulsoup4 lxml urllib3

## 2) Importações e configurações

In [2]:
import csv
import time
from datetime import datetime, timezone
from typing import List, Tuple, Set

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# Configurações básicas
G1_HOME = 'https://g1.globo.com/'
DEFAULT_TIMEOUT = 20
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0 Safari/537.36',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Referer': 'https://g1.globo.com/'
}
print('Configurações definidas.')

Configurações definidas.


## 3) Criar sessão HTTP com retries

In [3]:
def make_session() -> requests.Session:
    s = requests.Session()
    retry = Retry(total=5, backoff_factor=0.6, status_forcelist=[429, 500, 502, 503, 504], allowed_methods=frozenset(['GET','HEAD']))
    adapter = HTTPAdapter(max_retries=retry)
    s.mount('http://', adapter)
    s.mount('https://', adapter)
    s.headers.update(HEADERS)
    return s

SESSION = make_session()
print('Sessão criada com retries.')

Sessão criada com retries.


## 4) Funções utilitárias

In [4]:
def fetch_html(url: str) -> str:
    last_err = None
    for attempt in range(3):
        try:
            resp = SESSION.get(url, timeout=DEFAULT_TIMEOUT)
            resp.raise_for_status()
            return resp.text
        except requests.RequestException as e:
            last_err = e
            time.sleep(1.0 + attempt * 0.7)
    raise RuntimeError(f'Falha ao baixar URL {url}: {last_err}')

def normalize_url(base: str, href: str) -> str:
    if not href:
        return ''
    href = href.strip()
    if href.startswith('#') or href.lower().startswith('javascript:'):
        return ''
    abs_url = urljoin(base, href)
    parsed = urlparse(abs_url)
    if parsed.scheme not in {'http', 'https'}:
        return ''
    return abs_url

def parse_headlines(html: str, base_url: str) -> List[Tuple[str, str]]:
    try:
        soup = BeautifulSoup(html, 'lxml')
    except Exception:
        soup = BeautifulSoup(html, 'html.parser')
    candidates: List[Tuple[str, str]] = []

    def add_candidate(title: str, href: str):
        title = (title or '').strip()
        href = normalize_url(base_url, href)
        if title and href:
            candidates.append((title, href))

    for a in soup.select('a.feed-post-link'):
        title = a.get_text(strip=True)
        href = a.get('href')
        add_candidate(title, href)

    for h in soup.select('h1 a, h2 a, h3 a'):
        if 'feed-post-link' in (h.get('class') or []):
            continue
        title = h.get_text(strip=True)
        href = h.get('href')
        add_candidate(title, href)

    for a in soup.select('div.feed-post-body-title a, div.bastian-feed-item a'):
        title = a.get_text(strip=True)
        href = a.get('href')
        add_candidate(title, href)

    seen_links: Set[str] = set()
    seen_titles: Set[str] = set()
    unique: List[Tuple[str, str]] = []
    for title, link in candidates:
        if link in seen_links:
            continue
        if title.lower() in seen_titles:
            continue
        seen_links.add(link)
        seen_titles.add(title.lower())
        unique.append((title, link))
    return unique

def save_to_csv(rows: List[Tuple[str, str]], out_path: str):
    with open(out_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['title', 'link', 'fetched_at'])
        now = datetime.now(timezone.utc).isoformat()
        for title, link in rows:
            writer.writerow([title, link, now])
print('Funções definidas.')

Funções definidas.


## 5) Executar o scraper

In [6]:
URL_ALVO = G1_HOME  # Ex.: 'https://g1.globo.com/politica/'
LIMITE = 30         # 0 = sem limite

try:
    html = fetch_html(URL_ALVO)
    headlines = parse_headlines(html, URL_ALVO)
    if LIMITE and LIMITE > 0:
        headlines = headlines[:LIMITE]
    if not headlines:
        print('Nenhuma manchete encontrada.')
    else:
        print(f'Total coletado: {len(headlines)}')
        print('Manchetes principais (título + link):')
        for i, (title, link) in enumerate(headlines, start=1):
            print(f'{i:02d}. {title}')
            print(f'    {link}')
except Exception as e:
    headlines = []
    import traceback
    print('Erro durante a coleta:', e)
    traceback.print_exc()
len(headlines) if 'headlines' in globals() else 0

Total coletado: 9
Manchetes principais (título + link):
01. Questionada sobre Bolsonaro, Casa Branca diz que Trump não teme usar 'meios militares'
    https://g1.globo.com/mundo/noticia/2025/09/09/trump-nao-tem-medo-de-usar-meios-militares-para-proteger-liberdade-de-expressao-diz-casa-branca-sobre-eventual-condenacao-de-bolsonaro.ghtml
02. Mega-Sena pode pagar R$ 46 milhões; veja números
    https://g1.globo.com/loterias/mega-sena/noticia/2025/09/09/mega-sena-concurso-2912-resultado.ghtml
03. Dino: 'Alguém acredita que o Mickey vai mudar julgamento?'
    https://g1.globo.com/politica/noticia/2025/09/09/sera-que-alguem-acredita-que-um-tuite-de-presidente-estrangeiro-vai-mudar-o-julgamento-questiona-dino.ghtml
04. Placar do julgamento: como votou cada ministro até agora
    https://g1.globo.com/politica/noticia/2025/09/09/trama-golpista-placar-do-julgamento-infografico.ghtml
05. VÍDEOS: reveja o momento em que Moraes vota pela condenação
    https://g1.globo.com/politica/playlist/videos-

9

## 6) Salvar resultados em CSV

In [7]:
CAMINHO_CSV = 'g1_headlines.csv'
if 'headlines' in globals() and headlines:
    save_to_csv(headlines, CAMINHO_CSV)
    print(f'Salvo em: {CAMINHO_CSV}')
else:
    print('Nada para salvar: execute a coleta primeiro (célula 5).')

Salvo em: g1_headlines.csv
